<a href="https://colab.research.google.com/github/khushi200014/Danny-s-Pizza-Runner-Case-Study---SQL-/blob/main/Danny's_Pizza_Runner_Case_Study.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
files.download('pizza_runner.db')


In [ ]:
from google.colab import files
uploaded = files.upload()  # Upload your dataset CSV file


In [ ]:
import pandas as pd

# Assuming you uploaded 'orders.csv' with columns: order_id, order_date, order_time, delivery_time_minutes
df = pd.read_csv('orders.csv')

# Insert data from the DataFrame into the orders table
df.to_sql('orders', conn, if_exists='append', index=False)


In [1]:
import sqlite3
import pandas as pd

# Step 1: Connect to SQLite database (or create it if it doesn't exist)
conn = sqlite3.connect('pizza_runner.db')
cursor = conn.cursor()

# Step 2: Create the orders table
cursor.execute('''
    CREATE TABLE IF NOT EXISTS orders (
        order_id INTEGER PRIMARY KEY,
        order_date DATE,
        order_time TIME,
        delivery_time_minutes INTEGER
    )
''')

# Step 3: Insert sample data into the orders table
orders_data = [
    (1, '2024-03-01', '18:30:00', 30),
    (2, '2024-03-01', '19:15:00', 25),
    (3, '2024-03-02', '20:00:00', 40),
    (4, '2024-03-03', '21:30:00', 35),
    (5, '2024-03-04', '22:00:00', 20),
    (6, '2024-03-05', '18:45:00', 15),
    (7, '2024-03-06', '19:30:00', 50),
    (8, '2024-03-07', '20:15:00', 30),
    (9, '2024-03-08', '21:00:00', 40),
    (10, '2024-03-09', '18:30:00', 20)
]

cursor.executemany('''
    INSERT INTO orders (order_id, order_date, order_time, delivery_time_minutes)
    VALUES (?, ?, ?, ?)
''', orders_data)

conn.commit()

# Step 4: Query 1 - Analyzing delivery performance
query1 = '''
    SELECT
        COUNT(*) AS total_orders,
        AVG(delivery_time_minutes) AS avg_delivery_time
    FROM orders
'''
result1 = pd.read_sql_query(query1, conn)
print("Delivery Performance:\n", result1)

# Step 5: Query 2 - Identifying peak business hours
query2 = '''
    SELECT
        strftime('%H', order_time) AS hour_of_day,
        COUNT(*) AS orders_count
    FROM orders
    GROUP BY hour_of_day
    ORDER BY orders_count DESC
    LIMIT 5
'''
result2 = pd.read_sql_query(query2, conn)
print("\nPeak Business Hours:\n", result2)

# Step 6: Query 3 - Identifying the busiest day
query3 = '''
    SELECT
        CASE strftime('%w', order_date)
            WHEN '0' THEN 'Sunday'
            WHEN '1' THEN 'Monday'
            WHEN '2' THEN 'Tuesday'
            WHEN '3' THEN 'Wednesday'
            WHEN '4' THEN 'Thursday'
            WHEN '5' THEN 'Friday'
            WHEN '6' THEN 'Saturday'
        END AS day_of_week,
        COUNT(*) AS orders_count
    FROM orders
    GROUP BY strftime('%w', order_date)
    ORDER BY orders_count DESC
    LIMIT 1
'''
result3 = pd.read_sql_query(query3, conn)
print("\nBusiest Day:\n", result3)

# Step 7: Close the connection
conn.close()


Delivery Performance:
    total_orders  avg_delivery_time
0            10               30.5

Peak Business Hours:
   hour_of_day  orders_count
0          18             3
1          21             2
2          20             2
3          19             2
4          22             1

Busiest Day:
   day_of_week  orders_count
0      Friday             3
